# 1. Setup

## 1.1 - Packages

### 1.1.1 - Installations

In [0]:
# Core Data Handling
%pip install pandas --quiet
%pip install numpy --quiet
%pip install openpyxl --quiet

# Modeling and Statistical Analysis
%pip install statsmodels --quiet
%pip install pygam --quiet

# Actuarial Modeling
%pip install chainladder --quiet
%pip install sparse==0.15.5 --quiet  # Newer version conflicts with chainladder

# Performance and Parallel Processing
%pip install swifter --quiet
%pip install joblib --quiet
%pip install tqdm --quiet


### 1.1.2 - Imports

In [0]:
# Core Libraries
import pandas as pd
import numpy as np
from decimal import Decimal
from datetime import datetime
import itertools

# Visualization
import matplotlib.pyplot as plt

# Statistical Modeling
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pygam import PoissonGAM, GAM, s, f, te

# Actuarial Modeling
import chainladder as cl

# Performance and Parallel Processing
from tqdm import tqdm
from joblib import Parallel, delayed


## 1.2 - Parameterisation

In [0]:
# Parameters based on assumptions 
product = "Private Motor"
development_term = 24
last_day_previous_month = datetime(2025, 5, 31)
last_n_month_lognormal = 12
latest_balance_date_str = last_day_previous_month.strftime('%Y-%m-%d')

print(latest_balance_date_str)

# 2. Data Processing 

## 2.1 - Data Loading

In [0]:
# Main Data
data_claims = pd.read_feather('/Volumes/actuaries_prd/general/ibnr/data/claims_data.feather')
data_exposure = pd.read_feather('/Volumes/actuaries_prd/general/ibnr/data/exposure_data.feather')
data_gwp = pd.read_feather('/Volumes/actuaries_prd/general/ibnr/data/gwp_data.feather')
data = pd.read_feather('/Volumes/actuaries_prd/general/ibnr/data/data_claims_exposure.feather')

In [0]:
# Raw data saved to table:

# spark.createDataFrame(raw_data).write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("actuaries_prd.general.pm_ibnr_raw_data")

In [0]:
data.columns

In [0]:
raw_data = spark.sql(f"""
    SELECT 
        DATE_TRUNC('MM', g.loss_date) AS acc_month, 
        GREATEST(DATE_TRUNC('MM', a.observation_year_month),DATE_TRUNC('MM', g.loss_date)) AS obs_month,
        greatest((YEAR(a.observation_year_month) - YEAR(g.loss_date)) * 12 + MONTH(a.observation_year_month) - MONTH(g.loss_date) + 1,1 ) AS dev_month,
        CASE WHEN f.claim_code_mapped IS NULL THEN 'NAF' ELSE f.claim_code_mapped END AS claim_type,
        g.ANZO_Super_Class AS product_group,
        CASE WHEN coalesce(g.cell_name, 'Direct') = 'Private Motor' THEN 'Direct' else coalesce(g.cell_name, 'Direct') END AS channel,
        SUM(a.new_claims_count) AS claim_count, 
        SUM(a.net_claims_incurred_movement_amount_gst_excl) AS net_claim_incurred,
        SUM(a.gross_claims_incurred_movement_amount_gst_excl) as gross_claim_incurred,
        SUM(a.claim_recoveries_movement_amount_gst_excl) as recoveries
    FROM 
        cds_prd.cds.claim_claim_transactionmonth_financialcounts a 
    LEFT JOIN
        cds_prd.rds.claim_claim_transactiondaily_financialcounts_detail g on a.claim_fkey = g.claim_origin_key
    LEFT JOIN 
        ids_prd.ref.ref_cause_of_loss c ON a.cause_of_loss_fkey = c.origin_key
    LEFT JOIN
        actuaries_prd.general.pm_claimtype_mapping f on c.claim_code = f.claim_code
    LEFT JOIN 
        (select distinct PolicyNumber,ReferenceProductCode_Ext FROM staging_prd.gw.pc_policyperiod) h ON g.Policy_Number = h.PolicyNumber
    WHERE 
        YEAR(g.loss_date) >=2017
        AND g.anzo_super_class = '{product}'
        AND a.observation_year_month <= '{latest_balance_date_str}'
        AND g.loss_date <= '{latest_balance_date_str}'
        AND c.incident_description is not null
        AND coalesce(Distribution_Area, 'NULL') <> 'Motorcycle'
        AND (REPLACE(g.account_number, '||', '') not in ('AVMYTESLA', '56PORSCHN', '56PORSCHR','1Q0009136','1Q0009137','1Q0009138','1Q0009139','1Q0009140','1Q0009142','1Q0009162','1Q0009168','1Q0009178','40STELLA') or g.account_number is null)
        AND coalesce(ReferenceProductCode_Ext, 'NULL') <>'CVT'
        AND coalesce(claims_category) <>'Catastrophe'
    GROUP BY 
        all
    ORDER BY 
        all
    """).toPandas()


In [0]:
expo_data = spark.sql(f"""
WITH policy_transformed AS (
    SELECT DISTINCT 
        CONCAT(COMPANY_CODE, POLICY_BR, POLICY_NO, POLICY_TYP) AS Policy, 
        CONCAT(ACCOUNT_BR, ACCOUNT_NO) AS Account,
        ROW_NUMBER() OVER (PARTITION BY COMPANY_CODE, POLICY_BR, POLICY_NO, POLICY_TYP ORDER BY DATE_EFFECT DESC) AS rn
    FROM STAGING_PRD.EVO.tal_evo_polh_01
    WHERE ACCOUNT_BR IS NOT NULL AND ACCOUNT_NO IS NOT NULL
),
expo_data_raw AS (
    SELECT  
        DATE_TRUNC('MM', exp_start) AS acc_month,
        CASE 
            WHEN channel = 'ANZ' THEN 'ANZ'
            WHEN channel = 'BD' THEN 'Broker Distribution'
            WHEN channel = 'ELDERS' THEN 'Elders'
            WHEN channel = 'FIOTHER' THEN 'FI Other'
            WHEN channel = 'M TRADE' THEN 'Motor Trades'
            WHEN channel IN ('DIRECT', 'AUSPOST', 'KOGAN') THEN 'Direct'
        END AS channel,
        F_policyno AS policyno,
        earnprem AS earnprem,
        exposure AS exposure
    FROM actuarial_onprem_sqlserver.dbo.fact_mpa_prem
    WHERE exp_start >= '2017-01-01' AND exp_start <= '{latest_balance_date_str}'
)
SELECT e.acc_month, e.channel, SUM(e.earnprem) AS earnprem, SUM(e.exposure) AS exposure
FROM expo_data_raw e
LEFT JOIN policy_transformed p 
    ON e.policyno = p.Policy AND p.rn = 1
WHERE p.Account IS NULL OR p.Account NOT IN ('AVMYTESLA', '56PORSCHN', '56PORSCHR','1Q0009136','1Q0009137',
                      '1Q0009138','1Q0009139','1Q0009140','1Q0009142','1Q0009162',
                      '1Q0009168','1Q0009178','40STELLA')
GROUP BY e.acc_month, e.channel
order by 1,2
""").toPandas()

## 2.2 - Filtering

In [0]:
# Parameters
data_date = '2025-05-31'
development_period_end = 24
n_periods = 12
triangle_groups = ['channel', 'claim_type']
acc_month_start = pd.to_datetime('2017-01-01')

# Date Range
valuation_dates = pd.date_range(start="2019-01-31", end=data_date, freq='ME').strftime('%Y-%m-%d').tolist()

# Pre-filtering
input_data = data[
    (data['dev_month'] <= development_term) &
    (data['acc_month'] >= acc_month_start)
]

In [0]:
display(raw_data)

In [0]:
print(type(data['dev_month'].iloc[0]))
print(sorted(data['dev_month'].unique()))
print(sorted(input_data['dev_month'].unique()))


sample_data = input_data[
    (input_data['channel'] == 'Broker Distribution') &
    ((input_data['claim_type'] == 'AFM') + (input_data['claim_type'] == 'WND'))
]
display(sample_data)

# 3. Modelling

## 3.1 - Build Model

In [0]:
def build_models(input_data, valuation_date, development_period_end = 24, triangle_groups = ['channel', 'claim_type'], n_periods = 12):

    # 1. Data Processing
    # =================================

    data_hidden = input_data[
        (input_data['obs_month'] <= valuation_date) &
        (input_data['acc_month'] <= valuation_date)
    ] 

    data_full = input_data.copy()

    # Create triangle on partial data
    triangle_combined = cl.Triangle(
        data_hidden,
        origin="acc_month",
        development="obs_month",
        columns=[
            'claim_count',
            'net_claim_incurred',
            'gross_claim_incurred',
            'net_claim_incurred_indexed',
            'gross_claim_incurred_indexed',
            'recoveries_indexed',
            'earnprem_indexed',
            'exposure',
            'recoveries',
            'earnprem'
        ],
        index=triangle_groups,
        cumulative=False
    ).incr_to_cum()

    # Transformations
    triangle_combined['frequency'] = triangle_combined['claim_count'] / triangle_combined['exposure']
    triangle_combined['gross_cost_per_policy'] = triangle_combined['gross_claim_incurred'] / triangle_combined['exposure']
    triangle_combined['gross_cost_per_policy_indexed'] = triangle_combined['gross_claim_incurred_indexed'] / triangle_combined['exposure']
    triangle_combined['net_cost_per_policy'] = triangle_combined['net_claim_incurred'] / triangle_combined['exposure']
    triangle_combined['net_cost_per_policy_indexed'] = triangle_combined['net_claim_incurred_indexed'] / triangle_combined['exposure']
    triangle_combined['net_loss_ratio'] = triangle_combined['net_claim_incurred'] / triangle_combined['earnprem']
    triangle_combined['net_loss_ratio_indexed'] = triangle_combined['net_claim_incurred_indexed'] / triangle_combined['earnprem_indexed']

    # ------------------------------------
    # FULL TRIANGLE (FOR ACTUAL RESULTS)
    # ------------------------------------
    triangle_combined_full = cl.Triangle(
        data_full,
        origin='acc_month',
        development='obs_month',
        columns=[
            'claim_count',
            'net_claim_incurred',
            'gross_claim_incurred',
            'net_claim_incurred_indexed',
            'gross_claim_incurred_indexed',
            'recoveries_indexed',
            'earnprem_indexed',
            'exposure',
            'recoveries',
            'earnprem'
        ],
        index=triangle_groups,
        cumulative=False,
    ).incr_to_cum()

    output_actual_results = triangle_combined_full[
        [
            "net_claim_incurred",
            "gross_claim_incurred",
            "claim_count",
            "net_claim_incurred_indexed",
            "gross_claim_incurred_indexed",
            "recoveries_indexed",
        ]
    ].latest_diagonal.to_frame().reset_index().drop(
        columns=['valuation'] # Drop valuation and manually append after
    ).rename(
        columns={
            'origin' : 'acc_month',
            'claim_count': 'latest_view_claim_count',
            'net_claim_incurred': 'latest_view_net_claim_incurred',
            'gross_claim_incurred': 'latest_view_gross_claim_incurred',
            'recoveries': 'latest_view_recoveries',
            'net_claim_incurred_indexed': 'latest_view_net_claim_incurred_indexed',
            'gross_claim_incurred_indexed': 'latest_view_gross_claim_incurred_indexed',
            'recoveries_indexed': 'latest_view_recoveries_indexed'
        }
    )

    # 2.1 Model Training - Claim Count
    # =================================

    # Build initial development triangle and replace the development factors to assume fully developed in {development_period_end} periods
    claim_count_development_factors  = cl.Development(n_periods=n_periods).fit_transform(triangle_combined[['claim_count','frequency']])
    for i in range(0, claim_count_development_factors.ldf_.values.shape[0]):
        claim_count_development_factors.ldf_.values[i][:,0,development_period_end:] = 1
        claim_count_development_factors.cdf_.values[i][:,0,development_period_end:] = 1

    # ------------------------------------
    # MODEL 1 - Chainladder
    # ------------------------------------
    claim_count_chainladder = cl.Chainladder().fit(claim_count_development_factors)

    weights = triangle_combined['exposure'].latest_diagonal
    weights /= np.sum(weights, axis=2, keepdims=True)

    apriori_claim_count = np.sum((claim_count_chainladder.ultimate_['frequency'] * weights).iloc[:, :, -12:, :], axis=2, keepdims=True) / np.sum(weights.iloc[:, :, -12:, :], axis=2, keepdims=True)

    # ------------------------------------
    # MODEL 2 - Bornhuetter-Ferguson
    # ------------------------------------
    claim_count_bf = cl.BornhuetterFerguson(
        apriori=  apriori_claim_count
    ).fit(
        triangle_combined[['claim_count','exposure']]
        , sample_weight = triangle_combined['exposure'].latest_diagonal
    )

    # ------------------------------------
    # MODEL 3 - Cape Cod
    # ------------------------------------
    claim_count_cc = cl.CapeCod().fit(
        triangle_combined[['claim_count','exposure']],
        sample_weight=triangle_combined['exposure'].latest_diagonal
    )

    output_triangle = triangle_combined[
        [
            "exposure",
            "earnprem",
            "earnprem_indexed",
            "claim_count",
            "net_claim_incurred",
            "gross_claim_incurred",
            "recoveries",
            "net_claim_incurred_indexed",
            "gross_claim_incurred_indexed",
            "recoveries_indexed",
        ]
    ].latest_diagonal.to_frame().reset_index().drop(
        columns=['valuation']
    ).rename(
        columns={
            'origin' : 'acc_month',
            'claim_count': 'reported_to_date_claim_count',
            'net_claim_incurred': 'reported_to_date_net_claim_incurred',
            'gross_claim_incurred': 'reported_to_date_gross_claim_incurred',
            'recoveries': 'reported_to_date_recoveries',
            'net_claim_incurred_indexed': 'reported_to_date_net_claim_incurred_indexed',
            'gross_claim_incurred_indexed': 'reported_to_date_gross_claim_incurred_indexed',
            'recoveries_indexed': 'reported_to_date_recoveries_indexed'
        }
    )

    # 2.2 Model Training - Net Incurred
    # =================================

    # Build initial development triangle and replace the development factors to assume fully developed in {development_period_end} periods
    net_incurred_development_factors  = cl.Development(n_periods=n_periods).fit_transform(triangle_combined[['net_claim_incurred','net_cost_per_policy','net_loss_ratio']])
    for i in range(0, net_incurred_development_factors.ldf_.values.shape[0]):
        net_incurred_development_factors.ldf_.values[i][:,0,development_period_end:] = 1
        net_incurred_development_factors.cdf_.values[i][:,0,development_period_end:] = 1


    # ------------------------------------
    # MODEL 1 - Chainladder
    # ------------------------------------
    
    # Fit a chainladder model using the adjusted development factors from dev object
    net_incurred_chainladder = cl.Chainladder().fit(net_incurred_development_factors)

    weights = triangle_combined['earnprem'].latest_diagonal
    weights /= np.sum(weights, axis=2, keepdims=True)

    apriori_net_incurred = np.sum((net_incurred_chainladder.ultimate_['net_loss_ratio'] * weights).iloc[:, :, -12:, :], axis=2, keepdims=True) / np.sum(weights.iloc[:, :, -12:, :], axis=2, keepdims=True)

    # ------------------------------------
    # MODEL 2 - Bornhuetter-Ferguson
    # ------------------------------------

    net_incurred_bf = cl.BornhuetterFerguson(
        apriori=apriori_net_incurred
    ).fit(
        net_incurred_development_factors, 
        sample_weight=triangle_combined['earnprem'].latest_diagonal
    )

    # ------------------------------------
    # MODEL 3 - Cape Cod
    # ------------------------------------

    net_incurred_cc = cl.CapeCod().fit(
        net_incurred_development_factors,  # Use the same dev factors!
        sample_weight=triangle_combined['earnprem'].latest_diagonal
    )

    # 3. Joins to Final Dataframe
    # =================================

    # ------------------------------------
    # FULL TRIANGLE - ACTUAL RESULTS
    # ------------------------------------

    output_results = pd.merge(
        output_triangle,
        output_actual_results,
        left_on  = triangle_groups + ['acc_month'],
        right_on =  triangle_groups + ['acc_month'],
        how='left'
    ).drop(
        columns=['exposure']
    )

    # ------------------------------------
    # COUNT - CHAIN LADDER
    # ------------------------------------
    output_claim_count_chainladder = claim_count_chainladder.ultimate_.to_frame().reset_index(
    ).rename(
        columns={
            'claim_count': 'ultimate_claim_count_chainladder'
            ,'origin' : 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_claim_count_chainladder,
        left_on  = triangle_groups + ['acc_month'],
        right_on =  triangle_groups + ['acc_month'],
        how='outer'
    )

    # ------------------------------------
    # NET INCURRED - CHAIN LADDER
    # ------------------------------------

    output_net_incurred_chainladder = net_incurred_chainladder.ultimate_.to_frame().reset_index().drop(
        columns=['valuation', 'net_cost_per_policy']
    ).rename(
        columns={
            'net_claim_incurred': 'ultimate_net_incurred_chainladder',
            'origin': 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_net_incurred_chainladder,
        left_on = triangle_groups + ['acc_month'],
        right_on = triangle_groups + ['acc_month'],
        how='outer'
    )

    # ------------------------------------
    # COUNT - BORNHUETTER-FERGUSON
    # ------------------------------------
    output_claim_count_bf = claim_count_bf.ultimate_.to_frame().reset_index(
    ).rename(
        columns={
            'claim_count': 'ultimate_claim_count_bf'
            ,'origin' : 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_claim_count_bf,
        left_on  = triangle_groups + ['acc_month'],
        right_on =  triangle_groups + ['acc_month'],
        how='outer'
    )

    # ------------------------------------
    # NET INCURRED - BORNHUETTER-FERGUSON
    # ------------------------------------

    output_net_incurred_bf = net_incurred_bf.ultimate_.to_frame().reset_index().drop(
        columns=['valuation', 'net_cost_per_policy', 'net_loss_ratio']
    ).rename(
        columns={
            'net_claim_incurred': 'ultimate_net_incurred_bf',
            'origin': 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_net_incurred_bf,
        left_on = triangle_groups + ['acc_month'],
        right_on = triangle_groups + ['acc_month'],
        how='outer'
    )

    # ------------------------------------
    # COUNT - CAPE COD
    # ------------------------------------
    output_claim_count_cc = claim_count_cc.ultimate_.to_frame().reset_index().rename(
        columns={
            'claim_count': 'ultimate_claim_count_cc',
            'origin': 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_claim_count_cc,
        left_on=triangle_groups + ['acc_month'],
        right_on=triangle_groups + ['acc_month'],
        how='outer'
    )

    # ------------------------------------
    # NET INCURRED - CAPE COD
    # ------------------------------------

    output_net_incurred_cc = net_incurred_cc.ultimate_.to_frame().reset_index().drop(
        columns=['valuation', 'net_cost_per_policy', 'net_loss_ratio']
    ).rename(
        columns={
            'net_claim_incurred': 'ultimate_net_incurred_cc',
            'origin': 'acc_month'
        }
    )

    output_results = pd.merge(
        output_results,
        output_net_incurred_cc,
        left_on=triangle_groups + ['acc_month'],
        right_on=triangle_groups + ['acc_month'],
        how='outer'
    )
    
    # Final Output
    output_results['valuation_date'] = pd.to_datetime(valuation_date)

    # Aggregate required columns from the original data
    additional_fields = input_data.groupby(triangle_groups + ['acc_month'], as_index=False).agg({
        'product_group': 'first'  # Assuming 'product' is constant within each group
    })

    # Merge into the final output
    output_results = pd.merge(
        output_results,
        additional_fields,
        on=triangle_groups + ['acc_month'],
        how='left'
    )

    return output_results

## 3.2 - Parallel Runs

In [0]:
import warnings

# Suppress all warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Run in parallel
    valuation_data = Parallel(n_jobs=-1)(delayed(
        build_models
    )(
        sample_data,
        date,
        24,
        ['channel', 'claim_type'],
        12
    ) for date in tqdm(valuation_dates, desc="Processing valuation dates"))

    # Combine results into a single DataFrame
    combined_df = pd.concat(valuation_data, ignore_index=True)

combined_df

## 3.3 - Post-shaping

In [0]:
def reshape_forecast_output(
    df: pd.DataFrame,
    response_prefixes: dict = {
        'ultimate_claim_count': {
            'response_type': 'count',
            'actual_col': 'latest_view_claim_count',
            'reported_col': 'reported_to_date_claim_count'
        },
        'ultimate_net_incurred': {
            'response_type': 'net_incurred',
            'actual_col': 'latest_view_net_claim_incurred',
            'reported_col': 'reported_to_date_net_claim_incurred'
        }
    },
    id_columns: list = ['acc_month', 'valuation_date', 'channel', 'claim_type']
) -> pd.DataFrame:
    """
    Reshapes ultimate model output DataFrame into long format with actual and predicted values.
    Handles both claim count and net incurred amounts.

    Parameters:
    - df: Input DataFrame.
    - response_prefixes: Dictionary mapping response prefixes to their metadata including:
        - response_type: Label for type of response ('count', 'net_incurred', etc.)
        - actual_col: Column name representing actual value
        - reported_col: Column name representing reported-to-date value
    - id_columns: List of identifying columns to retain (e.g. ['acc_month', 'valuation_date', ...]).

    Returns:
    - A tidy DataFrame with columns: id_columns + ['model', 'actual', 'predicted', 'latest_view_*', 'reported_to_date_*', 'response']
    """
    result_dfs = []

    # Process each response prefix type
    for response_prefix, config in response_prefixes.items():
        response_type = config['response_type']
        actual_col = config['actual_col']
        reported_col = config['reported_col']
        
        # Skip if required columns are not in the DataFrame
        if actual_col not in df.columns or reported_col not in df.columns:
            continue
        
        # Identify ultimate columns for the given response prefix
        ultimate_cols = [col for col in df.columns if col.startswith(response_prefix)]
        
        if not ultimate_cols:
            continue
            
        models = [col.split('_')[-1] for col in ultimate_cols]

        for model, col_name in zip(models, ultimate_cols):
            # Make sure all required columns exist before proceeding
            required_cols = id_columns + [col_name, actual_col, reported_col]
            if not all(col in df.columns for col in required_cols):
                continue
                
            model_df = df[required_cols].copy()

            model_df['model'] = model
            model_df['actual'] = model_df[actual_col] - model_df[reported_col]
            model_df['predicted'] = model_df[col_name] - model_df[reported_col]
            model_df['response'] = response_type

            # Rename columns to standardised names to ensure they're consistent
            model_df = model_df.rename(columns={
                actual_col: 'latest_view',
                reported_col: 'reported_to_date'
            })

            # Drop the original ultimate column
            model_df = model_df.drop(columns=[col_name])
            result_dfs.append(model_df)

    # If no results were found, return empty DataFrame with correct columns
    if not result_dfs:
        return pd.DataFrame(columns=id_columns + ['model', 'actual', 'predicted', 'latest_view', 'reported_to_date', 'response'])
    
    # Combine all results
    result = pd.concat(result_dfs, ignore_index=True)
    
    # Ensure consistent column order
    result = result[id_columns + ['model', 'actual', 'predicted', 'latest_view', 'reported_to_date', 'response']]

    return result

In [0]:
# def reshape_forecast_output(
#     df: pd.DataFrame,
#     response_prefixes: dict = {
#         'ultimate_claim_count': {
#             'response_type': 'count',
#             'actual_col': 'latest_view_claim_count',
#             'reported_col': 'reported_to_date_claim_count'
#         },
#         'ultimate_net_incurred': {
#             'response_type': 'net_incurred',
#             'actual_col': 'latest_view_net_claim_incurred',
#             'reported_col': 'reported_to_date_net_claim_incurred'
#         }
#     },
#     id_columns: list = ['acc_month', 'channel', 'claim_type'],
#     additional_columns: list = ['exposure', 'earnprem', 'product_group']
# ) -> pd.DataFrame:
#     """
#     Reshapes ultimate model output DataFrame into long format with actual and predicted values.
#     Keeps extra business-related columns in final output.

#     Parameters:
#     - df: Input DataFrame.
#     - response_prefixes: Dictionary mapping response prefixes to their metadata.
#     - id_columns: List of identifying columns to retain.
#     - additional_columns: List of additional columns to retain in the reshaped output.

#     Returns:
#     - A tidy DataFrame with columns: id_columns + additional_columns + ['model', 'actual', 'predicted', 'latest_view', 'reported_to_date', 'response']
#     """
#     result_dfs = []

#     for response_prefix, config in response_prefixes.items():
#         response_type = config['response_type']
#         actual_col = config['actual_col']
#         reported_col = config['reported_col']
        
#         if actual_col not in df.columns or reported_col not in df.columns:
#             continue
        
#         ultimate_cols = [col for col in df.columns if col.startswith(response_prefix)]
#         if not ultimate_cols:
#             continue
            
#         models = [col.split('_')[-1] for col in ultimate_cols]

#         for model, col_name in zip(models, ultimate_cols):
#             required_cols = id_columns + [col_name, actual_col, reported_col]
#             optional_additional = [col for col in additional_columns if col in df.columns]
#             full_columns = required_cols + optional_additional

#             if not all(col in df.columns for col in required_cols):
#                 continue

#             model_df = df[full_columns].copy()
#             model_df['model'] = model
#             model_df['actual'] = model_df[actual_col] - model_df[reported_col]
#             model_df['predicted'] = model_df[col_name] - model_df[reported_col]
#             model_df['response'] = response_type

#             model_df = model_df.rename(columns={
#                 actual_col: 'latest_view',
#                 reported_col: 'reported_to_date'
#             })

#             model_df = model_df.drop(columns=[col_name])
#             result_dfs.append(model_df)

#     if not result_dfs:
#         return pd.DataFrame(columns=id_columns + ['model', 'actual', 'predicted', 'latest_view', 'reported_to_date', 'response'] + additional_columns)

#     result = pd.concat(result_dfs, ignore_index=True)

#     # Ensure consistent column order
#     optional_additional = [col for col in additional_columns if col in result.columns]
#     result = result[id_columns + optional_additional + ['model', 'actual', 'predicted', 'latest_view', 'reported_to_date', 'response']]

#     return result


In [0]:
result = reshape_forecast_output(
    df=combined_df,
    response_prefixes={
        'ultimate_claim_count': {
            'response_type': 'count',
            'actual_col': 'latest_view_claim_count',
            'reported_col': 'reported_to_date_claim_count'
        },
        'ultimate_net_incurred': {
            'response_type': 'net_incurred',
            'actual_col': 'latest_view_net_claim_incurred',
            'reported_col': 'reported_to_date_net_claim_incurred'
        }
    },
    id_columns=['acc_month', 'valuation_date', 'channel', 'claim_type']
)

In [0]:
result

## 3.4 - Method Selection

In [0]:
def calculate_mae_excl_last_6_months(data, actual_col, pred_cols, groupby_cols, date_col='valuation_date'):
    results = []

    for keys, group in data.groupby(groupby_cols):
        # Ensure date is datetime
        group[date_col] = pd.to_datetime(group[date_col])

        # Aggregate actual and predicted per valuation_date
        agg_list = {'valuation_date': group[date_col].unique()}
        agg_df = pd.DataFrame({'valuation_date': group[date_col]})
        agg_df[actual_col] = group[actual_col]
        for col in pred_cols:
            agg_df[col] = group[col]

        agg_df = agg_df.groupby('valuation_date').sum().reset_index()

        # Define cutoff date
        cutoff_date = agg_df['valuation_date'].max() - pd.DateOffset(months=6)
        filtered_df = agg_df[agg_df['valuation_date'] <= cutoff_date]

        # Compute MAE per model
        mae_dict = {}
        for col in pred_cols:
            mae = np.abs(filtered_df[actual_col] - filtered_df[col]).mean()
            mae_dict[col] = mae

        if not mae_dict:
            continue  # Skip if no models were evaluated

        # Find best model
        best_model = min(mae_dict, key=mae_dict.get)

        # Save results
        results.append({
            **dict(zip(groupby_cols, keys if isinstance(keys, tuple) else [keys])),
            **{f'mae_{col}': val for col, val in mae_dict.items()},
            'best_model': best_model
        })

    return pd.DataFrame(results)


### 3.4.1 - Claim Count

In [0]:
# Claim Count Method Selection
claim_count_best_method = calculate_mae_excl_last_6_months(
    combined_df,
    actual_col='latest_view_claim_count',
    pred_cols=[
        'ultimate_claim_count_chainladder',
        'ultimate_claim_count_bf',
        'ultimate_claim_count_cc'
    ],
    # groupby_cols=['channel', 'claim_type']
    groupby_cols=['channel'],
    date_col='valuation_date'
)

# Break down best_model into model and response
claim_count_best_method['response'] = 'count'
claim_count_best_method['model'] = claim_count_best_method['best_model'].str.split("_").str[-1]

claim_count_best_method

### 3.4.2 - Net Incurred

In [0]:
# Net Incurred Method Selection
net_incurred_best_method = calculate_mae_excl_last_6_months(
    combined_df,
    actual_col='latest_view_net_claim_incurred',
    pred_cols=[
        'ultimate_net_incurred_chainladder',
        'ultimate_net_incurred_bf',
        'ultimate_net_incurred_cc'
    ],
    # groupby_cols=['channel', 'claim_type']
    groupby_cols=['channel'],
    date_col='valuation_date'
)

# Break down best_model into model and response
net_incurred_best_method['response'] = 'net_incurred'
net_incurred_best_method['model'] = net_incurred_best_method['best_model'].str.split("_").str[-1]

net_incurred_best_method

# 4. Diagnostics

## 4.1 - Plotting Function

In [0]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import warnings

def plot_claims_AVE(output, mode='net_incurred', risk_class=None, channel=None, prem_class=None, claim_type=None, models=None):
    """
    Plot claims Average vs Expected with updated filtering options and MAE table.
    
    Parameters:
    -----------
    output : pandas DataFrame
        The dataset containing claims data
    mode : str, optional
        'count' or 'net_incurred', defaults to 'net_incurred'
    risk_class : str, optional
        Filter by specific risk class
    prem_class : str, optional
        Filter by specific premium class
    claim_type : str, optional
        Filter by specific claim type
    models : list, optional
        List of models to include
    """
    if mode not in ['count', 'net_incurred']:
        raise ValueError("Invalid mode. Available modes are: 'count', 'net_incurred'.")
    
    # Suppress warnings
    warnings.filterwarnings("ignore")
    
    # Filter data
    filtered_data = output[(output['response'] == mode)].copy()

    # Apply new filtering options
    if risk_class:
        filtered_data = filtered_data[filtered_data['risk_class'] == risk_class]
    if channel:
        filtered_data = filtered_data[filtered_data['channel'] == channel]
    if prem_class:
        filtered_data = filtered_data[filtered_data['prem_class'] == prem_class]
    if claim_type:
        filtered_data = filtered_data[filtered_data['claim_type'] == claim_type]
    if models:
        filtered_data = filtered_data[filtered_data['model'].isin(models)]
    
    # Convert balance month to datetime
    filtered_data['valuation_date'] = pd.to_datetime(filtered_data['valuation_date'])
    
    # Group by balance month and model, calculate sum of actual and predicted
    grouped_data = filtered_data.groupby(['valuation_date', 'model']).agg({
        'actual': 'sum',
        'predicted': 'sum'
    }).reset_index()
    
    # Extract actual values (taking first model's values for reference)
    if not grouped_data.empty and len(grouped_data['model'].unique()) > 0:
        first_model = grouped_data['model'].unique()[0]
        actual_values = grouped_data[grouped_data['model'] == first_model][['valuation_date', 'actual']]
    else:
        print("No data available for the selected filters.")
        return
    
    # Define cutoff date for MAE calculation (6 months from the latest date)
    latest_date = grouped_data['valuation_date'].max()
    cutoff_date = latest_date - pd.DateOffset(months=6)
    
    # Calculate MAE excluding last 6 months
    mae_results = {}
    for model in grouped_data['model'].unique():
        model_data = grouped_data[grouped_data['model'] == model]
        mae_data = model_data[model_data['valuation_date'] <= cutoff_date]
        if not mae_data.empty:
            mae = abs(mae_data['actual'] - mae_data['predicted']).mean()
            mae_results[model] = mae
    
    # Create a subplot with 2 rows - one for the chart and one for the table
    # Increase vertical spacing for more padding between chart and table
    fig = make_subplots(
        rows=2, 
        cols=1,
        row_heights=[0.75, 0.25],  # Adjusted to give more space to the table
        vertical_spacing=0.15,      # Increased for more padding
        specs=[[{"type": "scatter"}], [{"type": "table"}]]
    )
    
    # Add actual values trace to the first subplot
    fig.add_trace(
        go.Scatter(
            x=actual_values['valuation_date'], 
            y=actual_values['actual'], 
            mode='lines+markers', 
            name='Actual',
            line=dict(color='black', width=2),
            hovertemplate='%{x}<br>Actual: %{y:.2f}'
        ),
        row=1, col=1
    )
    
    # Define color mapping for consistent colors across models
    model_colors = {
        'chainladder': 'blue',
        'bornhuetter-ferguson': 'red',
        'cape-cod': 'green',
        'munich': 'purple'
    }
    
    # Add predicted values traces for each model to the first subplot
    for model in grouped_data['model'].unique():
        model_data = grouped_data[grouped_data['model'] == model]
        color = model_colors.get(model, None)  # Get color from mapping if available
        
        fig.add_trace(
            go.Scatter(
                x=model_data['valuation_date'], 
                y=model_data['predicted'], 
                mode='lines+markers', 
                name=f'ibnr - {model}',
                line=dict(color=color) if color else {},  # Apply color if defined
                hovertemplate='%{x}<br>Predicted: %{y:.2f}'
            ),
            row=1, col=1
        )
    
    # Build title with filtering information
    title_parts = [f'Claim {mode.capitalize()}: Actual vs Predicted']
    if risk_class:
        title_parts.append(f'Risk Class: {risk_class}')
    if channel:
        title_parts.append(f'Channel: {channel}')
    if prem_class:
        title_parts.append(f'Premium Class: {prem_class}')
    if claim_type:
        title_parts.append(f'Claim Type: {claim_type}')
    title = ' | '.join(title_parts)

    # Create a table for MAE results, rankings and filters
    # Sort models by MAE to determine ranking
    ranked_models = sorted([(model, mae) for model, mae in mae_results.items()], key=lambda x: x[1])
    
    # Create ordered lists for the table
    model_names = [model for model, _ in ranked_models]
    mae_values = [f"{mae:.2f}" for _, mae in ranked_models]
    # Create rankings (1 to n)
    rankings = [f"#{i+1}" for i in range(len(ranked_models))]
    
    # Prepare filter information for the table
    filter_names = []
    filter_values = []
    
    # Add mode to filter info
    filter_names.append("Mode")
    filter_values.append(mode)
    
    # Add other filters if they're set
    if risk_class:
        filter_names.append("Risk Class")
        filter_values.append(risk_class)
    if channel:
        filter_names.append("Channel")
        filter_values.append(channel)
    if prem_class:
        filter_names.append("Premium Class")
        filter_values.append(prem_class)
    if claim_type:
        filter_names.append("Claim Type")
        filter_values.append(claim_type)
    if models:
        filter_names.append("Models")
        filter_values.append(", ".join(models) if isinstance(models, list) else models)
    
    # Add cutoff date information
    filter_names.append("MAE Cutoff Date")
    filter_values.append(cutoff_date.strftime('%Y-%m-%d'))
    
    # Add MAE table to the second subplot - now including ranking column
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Ranking', 'Model', 'MAE (excluding last 6 months)', 'Filter', 'Value'],
                fill_color='paleturquoise',
                align='left',
                font=dict(size=12)
            ),
            cells=dict(
                values=[
                    rankings + [""] * (len(filter_names) - len(rankings)) if len(filter_names) > len(rankings) else rankings,
                    model_names + [""] * (len(filter_names) - len(model_names)) if len(filter_names) > len(model_names) else model_names,
                    mae_values + [""] * (len(filter_names) - len(mae_values)) if len(filter_names) > len(mae_values) else mae_values,
                    filter_names + [""] * (len(model_names) - len(filter_names)) if len(model_names) > len(filter_names) else filter_names,
                    filter_values + [""] * (len(model_names) - len(filter_values)) if len(model_names) > len(filter_values) else filter_values
                ],
                fill_color=[
                    ['lavender'] * len(rankings) + ['white'] * (len(filter_names) - len(rankings)) if len(filter_names) > len(rankings) else ['lavender'] * len(rankings),
                    ['lavender'] * len(model_names) + ['white'] * (len(filter_names) - len(model_names)) if len(filter_names) > len(model_names) else ['lavender'] * len(model_names),
                    ['lavender'] * len(mae_values) + ['white'] * (len(filter_names) - len(mae_values)) if len(filter_names) > len(mae_values) else ['lavender'] * len(mae_values),
                    ['ghostwhite'] * len(filter_names) + ['white'] * (len(model_names) - len(filter_names)) if len(model_names) > len(filter_names) else ['ghostwhite'] * len(filter_names),
                    ['ghostwhite'] * len(filter_values) + ['white'] * (len(model_names) - len(filter_values)) if len(model_names) > len(filter_values) else ['ghostwhite'] * len(filter_values)
                ],
                align='left',
                font=dict(size=11)
            )
        ),
        row=2, col=1
    )
    
    # Add zero line to the first subplot
    min_date = actual_values['valuation_date'].min()
    max_date = actual_values['valuation_date'].max()
    
    fig.add_shape(
        type="line",
        x0=min_date,
        y0=0,
        x1=max_date,
        y1=0,
        line=dict(color="gray", width=1, dash="dash"),
        row=1, col=1
    )
    
    # Update layout with more height and adjusted margins
    fig.update_layout(
        title=title,
        height=800,  # Increased height to better accommodate the expanded table
        margin=dict(t=100, b=50, l=50, r=50),
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update xaxis and yaxis for the first subplot
    fig.update_xaxes(
        title_text='Valuation Date',
        tickangle=-45,
        row=1, col=1
    )
    
    fig.update_yaxes(
        title_text=f'Claim {mode.capitalize()}',
        row=1, col=1
    )
    
    # Add annotation for best model at the bottom left of the chart
    if len(mae_results) > 1:
        best_model = ranked_models[0][0]
        best_mae = ranked_models[0][1]
        fig.add_annotation(
            x=0.07,  # Position at the bottom left (%)
            y=0.45,  # Position at the bottom left (%)
            xref="paper",
            yref="paper",
            text=f"Best model: {best_model} (MAE: {best_mae:.2f})",
            showarrow=False,
            font=dict(size=11, color="green"),
            align="left",
            bgcolor="rgba(255, 255, 255, 0.8)",  # Semi-transparent white background
            bordercolor="green",
            borderwidth=1,
            borderpad=4,
            xanchor="left",
            yanchor="bottom"
        )

    # fig.show()
    return fig

## 4.2 - Claim Count

In [0]:
# Get unique channels from the DataFrame
channels_list = result["channel"].dropna().unique()

# Collect all figures
claim_count_figures = []

# Loop through each channel and plot claim count
for channel in channels_list:
    fig = plot_claims_AVE(
        result,
        mode='count',
        channel=channel
    )
    claim_count_figures.append((channel, fig))

## 4.3 - Net Incurred

In [0]:
# Get unique channels from the DataFrame
channels_list = result["channel"].dropna().unique()

# Collect all figures
net_incurred_figures = []

# Loop through each channel and plot net incurred
for channel in channels_list:
    fig = plot_claims_AVE(
        result,
        mode='net_incurred',
        channel=channel
    )
    net_incurred_figures.append((channel, fig))

## 4.4 - Create HTML

In [0]:
import os

# Define output directory and ensure it exists
output_dir = "/Workspace/Shared/General/IBNR project/ibnr_modelling/outputs/claim_count_tabs"
os.makedirs(output_dir, exist_ok=True)

# Helper function to generate tabbed HTML sections
def generate_tab_section(figures, section_title, section_id_prefix):
    tabs_html = []
    buttons_html = []

    for i, (channel, fig) in enumerate(figures):
        chart_id = f"{section_id_prefix}_chart_{i}"
        chart_path = f"{output_dir}/{chart_id}.html"
        fig.write_html(chart_path, include_plotlyjs=False, full_html=False, div_id=chart_id)

        with open(chart_path, "r") as f:
            chart_div = f.read()

        tabs_html.append(f'<div id="{chart_id}_container" class="tabcontent" style="display:{"block" if i==0 else "none"};">{chart_div}</div>')
        buttons_html.append(f'<button class="tablinks" onclick="openTab(event, \'{chart_id}_container\')">{channel}</button>')

    section_html = f"""
    <h2>{section_title}</h2>
    <div class="tab">
      {''.join(buttons_html)}
    </div>
    {''.join(tabs_html)}
    """
    return section_html

# Generate HTML sections for claim count and net incurred figures
claim_count_section = generate_tab_section(claim_count_figures, "IBNR Claim Count AVE By Channel", "claim_count")
net_incurred_section = generate_tab_section(net_incurred_figures, "IBNR Net Incurred AVE By Channel", "net_incurred")

# Combine everything into a full HTML
full_html = f"""
<html>
<head>
<style>
.tab {{
  overflow: hidden;
  border-bottom: 1px solid #ccc;
}}
.tab button {{
  background-color: #f1f1f1;
  float: left;
  border: none;
  outline: none;
  cursor: pointer;
  padding: 10px 16px;
  transition: 0.3s;
  font-size: 14px;
}}
.tab button:hover {{
  background-color: #ddd;
}}
.tab button.active {{
  background-color: #ccc;
}}
.tabcontent {{
  display: none;
  padding: 10px;
  border-top: none;
}}
</style>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<script>
function openTab(evt, tabId) {{
  var i, tabcontent, tablinks;
  tabcontent = document.getElementsByClassName("tabcontent");
  for (i = 0; i < tabcontent.length; i++) {{
    tabcontent[i].style.display = "none";
  }}
  tablinks = document.getElementsByClassName("tablinks");
  for (i = 0; i < tablinks.length; i++) {{
    tablinks[i].className = tablinks[i].className.replace(" active", "");
  }}
  var tab = document.getElementById(tabId);
  tab.style.display = "block";
  evt.currentTarget.className += " active";

  var plots = tab.getElementsByClassName("plotly-graph-div");
  for (var j = 0; j < plots.length; j++) {{
      Plotly.Plots.resize(plots[j]);
  }}
}}
</script>
</head>
<body>

{claim_count_section}
<hr>
{net_incurred_section}

</body>
</html>
"""

# Save the final HTML file
final_html_path = "/Workspace/Shared/General/IBNR project/ibnr_modelling/temp_output.html"
with open(final_html_path, "w") as f:
    f.write(full_html)



In [0]:
# Read the HTML file content
with open("/Workspace/Shared/General/IBNR project/ibnr_modelling/temp_output.html", "r") as f:
    html_content = f.read()

# Display it in the notebook
displayHTML(html_content)

# 5. Ultimates Table

## 5.1 - Best Method Subset

In [0]:

# Union best method dataframes to be used for subsetting
best_method_union = pd.concat([claim_count_best_method, net_incurred_best_method]).drop_duplicates().reset_index(drop=True)

# Get best model results
bm_results = result.merge(
    best_method_union[['channel', 'model', 'response']],
    on=['channel', 'model', 'response'],
    how='inner'
)

# Retrieve latest valuation date for selected models
bm_results = bm_results[bm_results['valuation_date'] == latest_balance_date_str]
bm_results

# Check unique combinations of channel, response, and model
# unique_combinations = filtered_results[['channel', 'response', 'model']].drop_duplicates()
# unique_combinations


## 5.2 - Transform Output

### 5.2.1 - Merging

In [0]:
# Merge raw_actual with output_combined
ultimates_pre = pd.merge(
    raw_data.groupby(['acc_month', 'channel', 'claim_type'])[['claim_count', 'net_claim_incurred']].sum().reset_index(),
    bm_results,
    on=['acc_month', 'channel', 'claim_type'],
    how='outer'
)

# Merge with expo_data
ultimates_pre = pd.merge(ultimates_pre, expo_data, on=['acc_month', 'channel'])

# Fill missing values and convert to int64 for relevant columns
columns_to_fix = ['net_claim_incurred', 'claim_count', 'predicted',  'exposure']
ultimates_pre[columns_to_fix] = ultimates_pre[columns_to_fix].fillna(0).astype('int64')

ultimates_pre['product'] = product

### 5.2.2 - Pivoting

In [0]:
# Pivoting
pivot_indexes = ['acc_month', 'product', 'channel', 'claim_type', 'claim_count', 'net_claim_incurred', 'earnprem', 'exposure']

# Pivot the predicted values
predicted_pivot = ultimates_pre.pivot_table(index=pivot_indexes, 
                                 columns='response', 
                                 values='predicted').reset_index()
predicted_pivot.columns.name = None
predicted_pivot = predicted_pivot.rename(columns={
    'count': 'ibnr_count',
    'net_incurred': 'ibnr_incurred'
})

# Pivot the model values
model_pivot = ultimates_pre.pivot_table(index=pivot_indexes, 
                             columns='response', 
                             values='model', aggfunc='first').reset_index()
model_pivot.columns.name = None
model_pivot = model_pivot.rename(columns={
    'count': 'count_model',
    'net_incurred': 'incurred_model'
})

# Merge the two pivoted DataFrames
ultimates_df = pd.merge(predicted_pivot, model_pivot, on=pivot_indexes)

# Create ultimate count and ultimate incurred columns
ultimates_df['ultimate_count'] = ultimates_df['claim_count'] + ultimates_df['ibnr_count']
ultimates_df['ultimate_incurred'] = ultimates_df['net_claim_incurred'] + ultimates_df['ibnr_incurred']

display(ultimates_df)

### 5.2.3 - Inflation Adjustment

In [0]:
cpi_by_quarter = spark.sql(f"""
    SELECT 
        Date,
        CPI as cpi,
        CONCAT(YEAR(Date), 'Q', QUARTER(Date)) AS quarter
    FROM actuaries_prd.reference_data.abs_quarterly_cpi
""").toPandas()

In [0]:
test_df = ultimates_df.copy()

In [0]:
# Add Quarter to Ultimates Table
test_df['quarter'] = test_df['acc_month'].dt.to_period('Q').astype(str).str.replace('Q', 'Q', regex=False)

# CPI
ultimates_with_cpi = test_df.merge(cpi_by_quarter[['quarter', 'cpi']], on='quarter', how='left')

# Base CPI
base_cpi_quarter = sorted(ultimates_with_cpi['quarter'].unique())[-2] # Get second-last quarter in dataset
base_cpi_value = cpi_by_quarter.loc[cpi_by_quarter['quarter'] == base_cpi_quarter, 'cpi'].values[0]
ultimates_with_cpi['base_cpi'] = base_cpi_value

# Index Multiplier
ultimates_with_cpi['index_multiplier'] = ultimates_with_cpi['base_cpi'] / ultimates_with_cpi['cpi']

# Adjusted Incurreds
ultimates_with_cpi['adj_ultimate_incurred'] = ultimates_with_cpi['ultimate_incurred'] * ultimates_with_cpi['index_multiplier']
ultimates_with_cpi['adj_net_claim_incurred'] = ultimates_with_cpi['net_claim_incurred'] * ultimates_with_cpi['index_multiplier']

In [0]:
ultimates_with_cpi


### 5.2.4 - Update Table

In [0]:
# Save ultimates data down
# spark.createDataFrame(ultimates_df).write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("actuaries_prd.general.pm_ultimates_new")

In [0]:
# Save ultimates data down
spark.createDataFrame(ultimates_with_cpi).write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("actuaries_prd.general.pm_ultimates_new")